In [1]:
import csv

In [2]:
friends_chars={} 
Rachel=[]
Ross=[]
Joey=[]
Chandler=[]
Phoebe=[]
Monica=[]


with open("transcripts_friends/season_all/merged.csv", "r+") as fp:
    for cnt, line in enumerate(fp):
        #print("Line {}: {}".format(cnt, line))
        if line.startswith('Rachel:'):
            Rachel.append(line[8:])
        elif line.startswith('Ross:'):
            Ross.append(line[6:])
        elif line.startswith('Monica:'):
            Monica.append(line[8:])
        elif line.startswith('Chandler:'):
            Chandler.append(line[10:])
        if line.startswith('Phoebe:'):
            Phoebe.append(line[8:])
        if line.startswith('Joey:'):
            Joey.append(line[6:])

friends_chars['RACHEL']=Rachel
friends_chars['ROSS']=Ross
friends_chars['MONICA']=Monica
friends_chars['PHOEBE']=Phoebe
friends_chars['CHANDLER']=Chandler
friends_chars['JOEY']=Joey

In [3]:
import pandas as pd

df1 = pd.DataFrame(friends_chars['CHANDLER'])
df2 = pd.DataFrame(friends_chars['JOEY'])
df3 = pd.DataFrame(friends_chars['PHOEBE'])
df4 = pd.DataFrame(friends_chars['RACHEL'])
df5 = pd.DataFrame(friends_chars['ROSS'])
df6 = pd.DataFrame(friends_chars['MONICA'])

In [4]:
df1.head()

,0
0,"You know, I'd love to, but I'm a little tired.\n"
1,You're on!\n
2,He could be alone. This morning I heard him do...
3,She's cheating on Joey with Ross!\n
4,Wow!\n


In [5]:
listOfCharacters1 = [True] *df1.shape[0]
listOfCharacters2 = [False] *df2.shape[0]
listOfCharacters3 = [False] *df3.shape[0]
listOfCharacters4 = [False] *df4.shape[0]
listOfCharacters5 = [False] *df5.shape[0]
listOfCharacters6 = [False] *df6.shape[0]

In [6]:
df1['Chandler'] = listOfCharacters1
df2['Chandler'] = listOfCharacters2 
df3['Chandler'] = listOfCharacters3
df4['Chandler'] = listOfCharacters4
df5['Chandler'] = listOfCharacters5
df6['Chandler'] = listOfCharacters6

In [7]:
df1=df1.rename(columns={0: 'Dialogue'})
df2=df2.rename(columns={0: 'Dialogue'})
df3=df3.rename(columns={0: 'Dialogue'})
df4=df4.rename(columns={0: 'Dialogue'})
df5=df5.rename(columns={0: 'Dialogue'})
df6=df6.rename(columns={0: 'Dialogue'})

In [8]:
df = pd.concat([df1, df2,df3,df4,df5,df6])
df = df.sample(frac=1).reset_index(drop=True)

df.head()

,Dialogue,Chandler
0,I've gotta see this. All right Ugly Naked Guy!\n,False
1,"No, you should.\n",True
2,That might work! (She gives him the coat.)\n,False
3,(thinks) Four.\n,False
4,"Now that I untangled you, how 'bout you doing ...",True


In [9]:
df.head(20)

,Dialogue,Chandler
0,I've gotta see this. All right Ugly Naked Guy!\n,False
1,"No, you should.\n",True
2,That might work! (She gives him the coat.)\n,False
3,(thinks) Four.\n,False
4,"Now that I untangled you, how 'bout you doing ...",True
5,"Yes, it was!\n",False
6,(To Rachel) You completely stole my answer!\n,False
7,"Oh, thank you so much!\n",False
8,"Alright, alright, alright...\n",False
9,So you're gonna have to do it in the mess!\n,False


In [10]:
from io import StringIO
col = [ 'Dialogue','Chandler']
df = df[col]
df = df[pd.notnull(df['Dialogue'])]
df.columns = ['Dialogue', 'Chandler']
category_id_df = df[['Dialogue', 'Chandler']]
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['Chandler', 'Dialogue']].values)
df.head(10)

,Dialogue,Chandler
0,I've gotta see this. All right Ugly Naked Guy!\n,False
1,"No, you should.\n",True
2,That might work! (She gives him the coat.)\n,False
3,(thinks) Four.\n,False
4,"Now that I untangled you, how 'bout you doing ...",True
5,"Yes, it was!\n",False
6,(To Rachel) You completely stole my answer!\n,False
7,"Oh, thank you so much!\n",False
8,"Alright, alright, alright...\n",False
9,So you're gonna have to do it in the mess!\n,False


In [11]:
import re
from nltk.corpus import stopwords


df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['Dialogue'] = df['Dialogue'].apply(clean_text)
df['Dialogue'] = df['Dialogue'].str.replace('\d+', '')
df.head()

,Dialogue,Chandler
0,ive gotta see right ugly naked guy,False
1,,True
2,might work gives coat,False
3,thinks four,False
4,untangled bout little something,True


In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [22]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 20000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 120
# This is fixed.
EMBEDDING_DIM = 64
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1['Dialogue'].values)
word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index))
#word_index

In [14]:
X = tokenizer.texts_to_sequences(df['Dialogue'].values)#df['Dialogue']#tokenizer.texts_to_sequences(df['Dialogue'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)
X

Shape of data tensor: (99233, 120)


array([[   0,    0,    0, ..., 1004,  420,  119],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,  175,  653, 1315],
       ...,
       [   0,    0,    0, ...,    0,   15, 3055],
       [   0,    0,    0, ...,   15,  123, 1953],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int32)

In [15]:
Y = pd.get_dummies(df['Chandler']).values#df['category_id']
#print('Shape of label tensor:', Y.shape)
Y

array([[1, 0],
       [0, 1],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 0)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(66486, 120) (66486, 2)
(32747, 120) (32747, 2)


In [17]:
X_test

array([[   0,    0,    0, ..., 2035,   94,   84],
       [   0,    0,    0, ...,   15,  329, 6026],
       [   0,    0,    0, ...,  160,  387,  300],
       ...,
       [   0,    0,    0, ...,  116,  229, 5434],
       [   0,    0,    0, ...,    0,   31,   77],
       [   0,    0,    0, ...,    0,   39,  241]], dtype=int32)

In [18]:
Y_test

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [19]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [20]:
X.shape[1]

120